In [ ]:
import tensorflow as tf

In [ ]:

import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import time
import re
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
random.seed(123)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df_train = pd.read_csv("A.csv")
df_dev = pd.read_csv("A.csv")
df_test = pd.read_csv("A.csv")

In [ ]:
X_train = df_train['commentaire']

In [ ]:
X_dev = df_dev['commentaire']
X_test  = df_test['commentaire']

In [ ]:
y_train = df_train['note']
y_dev = df_dev['note']

In [ ]:
tokenizer = Tokenizer(num_words=len(list(X_train.unique())))

In [ ]:
tokenizer.fit_on_texts(list(X_train))

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train) 
X_dev = tokenizer.texts_to_sequences(X_dev)

In [ ]:
l = [len(item) for item in X_train ]
sns.distplot(l, hist=True, kde=True, 
             bins=int(50), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
len_max = 180

In [ ]:
X_train = pad_sequences(X_train, maxlen=len_max)
X_dev = pad_sequences(X_dev, maxlen=len_max)

In [ ]:

unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
print(len(list(unique_words)))
print(len_max)

In [ ]:
len(list(unique_words)),len_max

In [ ]:
num_classes = len(y_train.unique())
num_classes

In [ ]:
model=Sequential()
model.add(Embedding(len(list(unique_words)),100,input_length=180))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(256,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(5e-3),metrics=['accuracy'])
model.summary()

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'my_best_model.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model.fit(X_train, y_train, validation_data=(X_dev, y_dev),epochs=5, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))